In [ ]:
from typing import Any

from ndv.data import cells3d
from ndv.models import LUTModel, StatsModel
from ndv.views import get_histogram_backend
from ndv.views.protocols import PHistogramView


# TODO: Put this somewhere else.
class Controller:
    """A (Qt) wrapper around another HistogramView with some additional controls."""

    def __init__(
        self,
        stats_model: StatsModel | None = None,
        lut_model: LUTModel | None = None,
        view: PHistogramView | None = None,
    ) -> None:
        if stats_model is None:
            stats_model = StatsModel()
        if lut_model is None:
            lut_model = LUTModel()
        if view is None:
            view = get_histogram_backend()
        self._stats = stats_model
        self._lut = lut_model
        self._view = view

        # A HistogramView is both a StatsView and a LUTView
        # StatModel <-> StatsView
        self._stats.events.data.connect(self._set_data)
        self._stats.events.bins.connect(self._set_data)
        # LutModel <-> LutView
        self._lut.events.clims.connect(self._set_model_clims)
        self._view.climsChanged.connect(self._set_view_clims)
        self._lut.events.gamma.connect(self._set_model_gamma)
        self._view.gammaChanged.connect(self._set_view_gamma)

    def _set_data(self) -> None:
        values, bin_edges = self._stats.histogram
        self._view.setHistogram(values, bin_edges)

    def _set_model_clims(self) -> None:
        clims = self._lut.clims
        self._view.setClims(clims)

    def _set_view_clims(self, clims: tuple[float, float]) -> None:
        self._lut.clims = clims

    def _set_model_gamma(self) -> None:
        gamma = self._lut.gamma
        self._view.setGamma(gamma)

    def _set_view_gamma(self, gamma: float) -> None:
        self._lut.gamma = gamma

    def view(self) -> Any:
        """Returns an object that can be displayed by the active backend."""
        return self._view


viewer = Controller()
viewer._stats.data = cells3d()

viewer.view().show()

In [ ]:
# Change the data
from numpy.random import normal

viewer._stats.data = normal(30000, 10000, 10000000)